# Laboratorio 2025
### Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1


## Importación de módulos

In [ ]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues, Alarm
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
from pydistsim.restrictions.topological import Connectivity
from pydistsim.restrictions.knowledge import InitialDistinctValues

from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger, disable_logger, logger

from pydistsim.gui import drawing as draw
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
from pprint import pformat

import time

from utils import *

set_log_level(LogLevels.INFO)
# disable_logger()
enable_logger()

## Parte 1: 

### 1. Implementación básica del problema

- Simule un conjunto de n generales (nodos) que deben decidir si atacar o retirarse.
- Permita que hasta f generales sean bizantinos, es decir, que puedan enviar mensajes contradictorios.
- Los generales leales deberán acordar una decisión común, cumpliendo las condiciones de consistencia y validez.

### 2. Protocolos de comunicación


- Implemente el protocolo recursivo propuesto por Lamport, Shostak y Pease (OM(f)).

In [ ]:
class ByzantineGenerals(NodeAlgorithm):
    def __init__(self, simulation, *args, f=None, verbose=False, **kwargs):
        super().__init__(simulation, *args, **kwargs)
        self.f = f
        self.siege = Siege(len(self.network.nodes()) - f)
        self.messages_counter = 0
        self.logging = []
        # self.max_messages = self.network.size() ** (f + 2)

    default_params = {
        "Value" : "Value",
    }

    class Status(StatusValues):
        COMMANDER = "COMMANDER"
        GENERAL = "GENERAL"
        DONE = "DONE"
        TRAITOR = "TRAITOR"
    S_init = (Status.COMMANDER, Status.GENERAL, Status.TRAITOR)
    S_term = (Status.DONE,)

    algorithm_restrictions = (
        BidirectionalLinks,
        Connectivity, 
        TotalReliability, # 3.A1 TODO no asumirlo?? ver notas
        InitialDistinctValues 
    )
        
    def initializer(self):
        self.apply_restrictions()
        traidores = random.sample(list(self.network.nodes()), self.f)
        commander_general = self.network.nodes_sorted()[0]
        for node in self.network.nodes():
            node.memory["values_tree"] = {}
            node.memory["decision_tree"] = {}
            node.memory["liutenants"] = self.network.nodes() - {node, commander_general}
            node.status = self.Status.GENERAL if node not in traidores else self.Status.TRAITOR
        if commander_general not in traidores:
            commander_general.status = self.Status.COMMANDER
        commander_general.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=commander_general))
 

    def get_included_liutenants(self, node: NodeAccess, path):
        already_visited = []
        for neigh in node.memory["liutenants"]:
            if neigh.memory["unique_value"] in path:
                already_visited.append(neigh)
        # self.logging.append(f[General {node.memory["unique_value"]}] de {node.memory["liutenants"]}, descarto {already_visited}")
        return node.memory["liutenants"] - set(already_visited)

    def OM_commander(self, node : NodeAccess, packet : Data):
        liutenants = self.get_included_liutenants(node, packet.path)
        # self.logging.append(f"[General {node.memory['unique_value']}] Enviando mensaje con data {packet} por el path {packet.path} a los tenientes {liutenants}")
        send_with_check(
            node, 
            datos=packet, 
            dest=liutenants,
            msj_type=self.default_params["Value"],
            algorithm=self,
        )



    def checks(self, node: NodeAccess, message = None):
        pass
        # self.logging.append("\n\n")
        # if message:
        #     datos : Data = message.data
        #     self.logging.append(f"[General {node.memory['unique_value']}] Le llega el mensaje con data {datos}")                        
        # if node.status == self.Status.TRAITOR:
        #     self.logging.append(f"[Traidor {node.memory['unique_value']}]")
        # elif node.status == self.Status.COMMANDER:
        #     self.logging.append(f"\n\n[General {node.memory['unique_value']}] Observes they should {node.memory['values_tree'][()]}")

        # if self.messages_counter > self.max_messages:
        #     raiseError("Cota superior de mensajes excedida. Imprima logging para mas detalles.")

    def get_layer_decisions(self, node: NodeAccess, path):
        layer = len(path)
        # m = self.f - layer
        nodos = {key: value for (key, value) 
                in node.memory["decision_tree"].items() 
                if (len(key) == layer and key[:layer-1] == path[:layer-1])
                }
        ronda = nodos.values()
        awaiting_values = len(self.get_included_liutenants(node,path)) + 2 #TODO depende de si estoy en la uppermost
        # self.logging.append(f"[General {node.memory['unique_value']}] En el path {path}, esperando {awaiting_values} valores, recibidos {len(ronda)}: {list(ronda)}")
        if len(ronda) == (awaiting_values):
            return ronda
        return []


    @Status.COMMANDER
    def spontaneously(self, node: NodeAccess, _: Message):
        decision = node.memory["values_tree"][()] = self.siege.observe(node)
        self.checks(node)
        datos = Data((), decision)
        self.OM_commander(node, datos)
        if decision == GeneralDecision.ATTACK:
            self.siege.attack()
            node.status = self.Status.DONE
        else:
            self.siege.retreat()
            node.status = self.Status.DONE


    @Status.COMMANDER
    def receiving(self, node: NodeAccess, message: Message):
        error("COMMANDER::receiving", message)


    @Status.TRAITOR
    def spontaneously(self, node: NodeAccess, _: Message):
        self.checks(node)
        TraitorActions.send_confusing_signal(node, self, (), self.get_included_liutenants(node, ()))
        node.status = self.Status.DONE

    @Status.TRAITOR
    def receiving(self, node: NodeAccess, message: Message):
        self.checks(node, message)
        if self.siege.state != Siege.State.ONGOING:
            node.status = self.Status.DONE
            return
        #Por ahora solo envia, no le importa nada del tamaño del path ni nada
        path = message.data.path
        new_path =  path + (node.memory["unique_value"],)
        TraitorActions.send_confusing_signal(node, self, new_path, self.get_included_liutenants(node, path))


    @Status.GENERAL
    def receiving(self, node: NodeAccess, message: Message):
        #TODO: chequear si tiene mi id en -2: si lo tiene el sender es malicioso. Descarto de mis liutenants
        self.checks(node, message)
        datos : Data = message.data
        path = datos.path
        m = self.f - len(path)
        if m > 0:
            recursive_path = path + (node.memory["unique_value"],)
            if len(recursive_path) > self.f:
                return
            node.memory["decision_tree"][recursive_path] = datos.value
            new_datos = Data(recursive_path, datos.value)
            self.OM_commander(node, new_datos)
        elif m == 0:
            node.memory["decision_tree"][path] = datos.value
            ronda = self.get_layer_decisions(node, path)
            while path != () and len(ronda) != 0: 
                maj = majority(ronda)
                path = path[:len(path)-1]
                node.memory["decision_tree"][path] = maj
                # self.logging.append(f"[General {node.memory['unique_value']}] Decision en path {path}: {maj}")
                ronda = self.get_layer_decisions(node, path)
            if path == ():
                # self.logging.append(f"Arbol de decisiones del General {node.memory['unique_value']}:\n{pformat(node.memory['decision_tree'], width=80, sort_dicts=True)}")
                maj = node.memory["decision_tree"][()]
                if maj == GeneralDecision.ATTACK:
                    self.siege.attack()
                    node.status = self.Status.DONE
                else:
                    self.siege.retreat()
                    node.status = self.Status.DONE


    @Status.DONE
    def default(self, *args, **kwargs):    
        pass

- Mida el número total de mensajes intercambiados y el tiempo necesario para llegar al consenso.

In [ ]:
def get_no_iters(layer):
    if layer == 1:
        return 10
    elif layer == 2:
        return 10
    elif layer == 3:
        return 10
    elif layer == 4:
        return 1
    else:
        return 1

for layer in range(1,4):
    print(f"Corriendo capa {layer}")
    f = layer
    n = 3*f + 1
    its = get_no_iters(layer)
    for i in range(its):
        logger.info(f"Iteracion {i}")
        net_gen = NetworkGenerator(directed=False)
        net = net_gen.generate_complete_network(n)
        sim = Simulation(net, check_restrictions=True)
        sim.algorithms = ((ByzantineGenerals, {"f": f, "verbose" : True}),)
        sim.reset()
        begin = time.perf_counter()
        sim.run()
        end = time.perf_counter()
        logger.info(f"Tiempo de ejecucion: {end - begin} segundos")
        alg = sim.algorithms[0]
        if alg.siege.state not in {Siege.State.SUCCESSFUL, Siege.State.ABORTED}:
            with open(f"results/f={layer}_it={i}.txt", "w") as res:
                for log in alg.logging:
                    print(log, file=res)
        logger.info(f"Se intercambiaron {alg.messages_counter} mensajes")
        logger.info("\n\n\n")


### 3. Análisis

- Determine experimentalmente el número mínimo de nodos necesario para alcanzar consenso frente a f fallos bizantinos.
- Verifique la condición teórica n ≥ 3f + 1.

In [ ]:
def find_minimum_generals_for_consensus(f : int, its : int = None):
    print(f"------------------------------------------------ Buscando n minimo para f = {f} ------------------------------------------------")
    protective_max = 4*f
    n_search = f
    statistic_success = False
    while not statistic_success and n_search <= protective_max:
        n_search +=1
        print(f"Probando con n = {n_search}")
        statistic_success = True
        for i in range(its):
            net_gen = NetworkGenerator(directed=False)
            net = net_gen.generate_complete_network(n_search)
            sim = Simulation(net, check_restrictions=True)
            sim.algorithms = ((ByzantineGenerals, {"f": f, "verbose" : True}),)
            sim.run()
            alg = sim.algorithms[0]
            statistic_success &= alg.siege.state in {Siege.State.SUCCESSFUL, Siege.State.ABORTED}
            if not statistic_success:
                break
    print("---------------------------------------------------------------------------------------------------------------------------------")
    return n_search if statistic_success else None


disable_logger()
fs = np.array([1, 2, 3])

for fi in fs:
    n_found = find_minimum_generals_for_consensus(fi, 50)
    if not n_found:
        print(f"No se encontro n minimo para f = {fi} en el espacio de busqueda permitido\n")
    elif n_found >= 3*fi + 1:
        print(f"Se cumple la condicion para f = {fi}: {n_found} ≥ 3*{fi} + 1\n")
    else:
        print(f"No se cumple la condicion para f = {fi}: {n_found} < 3*{fi} + 1\n")
enable_logger()

## Parte 2: Protocolos de Commit de Transacciones

### 1. Implementación del protocolo 2PC (Two-Phase Commit)

- Simule el proceso de coordinación entre un coordinador y varios participantes.

In [ ]:
class TwoPhaseCommit(NodeAlgorithm):
    def __init__(self, simulation, waiting_time = 15,*args, **kwargs):
        super().__init__(simulation, *args, **kwargs)
        self.transaction = Transaction(len(self.network.nodes()) - 1)
        self.messages_counter = 0
        self.logging = []
        self.waiting_time = waiting_time
        
    default_params = {
        "Prepare" : "Prepare", #Can you commit this transaction?
        "OK" : "OK", #I can commit the transaction
        "Commit" : "Commit", #Commit the transaction
        "Abort" : "Abort", #Abort the transaction
        "PrepareTimeout" : "PrepareTimeout", #Timeout waiting for participant's OKs
    }

    class Status(StatusValues):
        COORDINATOR = "COORDINATOR"
        PARTICIPANT = "PARTICIPANT"
        POST_PREPARE_COORDINATOR = "POST_PREPARE_COORDINATOR"
        POST_PREPARE_PARTICIPANT = "POST_PREPARE_PARTICIPANT"
        SUCCESS = "SUCCESS"
        CRASHED = "CRASHED"
    S_init = (Status.PARTICIPANT, Status.COORDINATOR,)
    S_term = (Status.SUCCESS, Status.CRASHED)

    algorithm_restrictions = (
        BidirectionalLinks,
        Connectivity, 
        #TotalReliability, #Pueden perderse mensajes
        InitialDistinctValues 
    )
        
    def initializer(self):
        self.apply_restrictions()
        coordinator = self.network.nodes_sorted()[0]
        for node in self.network.nodes():
            node.memory["coordinator"] = coordinator
            node.memory["crash_chance"] = CrashBehavior.crash_chance(node)
            node.status = self.Status.PARTICIPANT
        coordinator.memory["oks"] = {}
        coordinator.status = self.Status.COORDINATOR
        coordinator.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=coordinator))



    def crash(self, node : NodeAccess):
        self.logging.append(f"[Node {node.memory['unique_value']}] Crashed")
        if node.status != self.Status.COORDINATOR:
            self.transaction.crash()
        node.status = self.Status.CRASHED

    def send_to_participants(self, node : NodeAccess, header: str):
        for neighbor in node.neighbors():
            if CrashBehavior.determine_crash(node):
                self.crash(node)
                return
            self.logging.append(f"[Node {node.memory['unique_value']}] Sending '{header}' to [Node {neighbor}]")
            self.send(
                node, 
                data=None,
                destination=neighbor,
                header=header
            )

    @Status.COORDINATOR
    def spontaneously(self, node: NodeAccess, _: Message):
        self.logging.append(f"[Node {node.memory['unique_value']}] is the coordinator")
        self.send_to_participants(node, self.default_params["Prepare"])
        m = Message(
            destination=node.memory["coordinator"],
            header=self.default_params["PrepareTimeout"]
            )
        node.memory["PrepareAlarm"] = self.set_alarm(node, self.waiting_time, m)
        node.status = self.Status.POST_PREPARE_COORDINATOR


    @Status.POST_PREPARE_COORDINATOR
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["OK"]:
            node.memory["oks"][message.source] = True
            if len(node.memory["oks"]) == len(node.neighbors()):
                self.disable_alarm(node.memory["PrepareAlarm"])
                self.send_to_participants(node, self.default_params["Commit"])
                node.status = self.Status.SUCCESS
        else:
            error("POST_PREPARE_COORDINATOR::receiving", message)
        

    @Status.POST_PREPARE_COORDINATOR
    def alarm(self, node: NodeAccess, message: Message):
        if node.status != self.Status.POST_PREPARE_COORDINATOR:
            self.logging.append("Why the fuck is the coordinator receiving this message if he is no longer in the post_prepare_coordinator stage??")
        if message.header == self.default_params["PrepareTimeout"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Coordinator not waiting any longer for OKs, aborting transaction")
            self.send_to_participants(node, self.default_params["Abort"])
            node.status = self.Status.SUCCESS
        else:
            error("POST_PREPARE_COORDINATOR::receiving", message)
        


    @Status.PARTICIPANT
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["Prepare"]:
            if CrashBehavior.determine_crash(node):
                self.crash(node)
                return
            self.send(
                node, 
                data=None,
                destination=node.memory["coordinator"],
                header=self.default_params["OK"]
            )
            node.status = self.Status.POST_PREPARE_PARTICIPANT
        elif message.header == self.default_params["Abort"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Aborting transaction")
            self.transaction.abort()
            node.status = self.Status.SUCCESS
        else:
            error("PARTICIPANT::receiving", message)


    @Status.POST_PREPARE_PARTICIPANT
    def receiving(self, node: NodeAccess, message: Message):
        #Asumo que los participantes no fallan despues de confirmar que pueden hacer la transaccion
        if message.header == self.default_params["Commit"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Committing transaction")
            self.transaction.commit()
            node.status = self.Status.SUCCESS
        elif message.header == self.default_params["Abort"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Aborting transaction")
            self.transaction.abort()
            node.status = self.Status.SUCCESS
        else:
            error("POST_PREPARE_PARTICIPANT::receiving", message)


    @Status.SUCCESS
    def default(self, *args, **kwargs):    
        pass

    @Status.CRASHED
    def default(self, *args, **kwargs):    
        pass


#### Pruebas

Forzar crash en momentos concretos:

In [ ]:
n = 5
for i in range(10):
    net_gen = NetworkGenerator(directed=False)
    net = net_gen.generate_complete_network(n)
    sim = Simulation(net, check_restrictions=True)
    sim.algorithms = ((TwoPhaseCommit, {}),)
    sim.reset()
    sim.run()
    alg = sim.algorithms[0]
    for log in alg.logging:
        logger.info(log)
    if alg.transaction.state == Transaction.TransactionState.INCONSISTENT:
        logger.info(f"Iteracion {i} perdió atomicidad")
    elif alg.transaction.state == Transaction.TransactionState.ONGOING:
        logger.info(f"Iteracion {i} terminó en deadlock")
    else:
        logger.info(f"Iteracion {i} terminó correctamente con estado {alg.transaction.state}")
    logger.info("\n")

### 2. Extensión al protocolo 3PC (Three-Phase Commit)

- Mejore el protocolo anterior incorporando una fase intermedia para evitar bloqueos permanentes.

In [ ]:
class ThreePhaseCommit(NodeAlgorithm):
    def __init__(self, simulation, waiting_time = 15,*args, **kwargs):
        super().__init__(simulation, *args, **kwargs)
        self.transaction = Transaction(len(self.network.nodes()) - 1)
        self.messages_counter = 0
        self.logging = []
        self.waiting_time = waiting_time
        
    default_params = {
        "Prepare" : "Prepare", #Can you commit this transaction?
        "OK" : "OK", #I can commit the transaction
        "PreCommit" : "PreCommit", #Get ready for commiting the transaction
        "Ready" : "Ready", #I'm ready for commiting the transaction
        "Commit" : "Commit", #Commit the transaction
        "Abort" : "Abort", #Abort the transaction
        "PrepareTimeout" : "PrepareTimeout", #Timeout waiting for participant's OKs
        "PreCommitTimeout" : "PreCommitTimeout", #Timeout waiting for coordinator's PreCommit
        "DoCommitTimeout" : "DoCommitTimeout", #Timeout waiting for coordinator's DoCommit

        #TODO implementar comunicacion vecinos
    }

    class Status(StatusValues):
        COORDINATOR = "COORDINATOR"
        PARTICIPANT = "PARTICIPANT"
        POST_PREPARE_COORDINATOR = "POST_PREPARE_COORDINATOR" 
        POST_PREPARE_PARTICIPANT = "POST_PREPARE_PARTICIPANT" 
        PRE_COMMIT_COORDINATOR = "PRE_COMMIT_COORDINATOR"
        PRE_COMMIT_PARTICIPANT = "PRE_COMMIT_PARTICIPANT"
        RECOVERING = "RECOVERING"
        SUCCESS = "SUCCESS"
        CRASHED = "CRASHED"
    S_init = (Status.PARTICIPANT, Status.COORDINATOR,)
    S_term = (Status.SUCCESS, Status.CRASHED)

    algorithm_restrictions = (
        BidirectionalLinks,
        Connectivity, 
        #TotalReliability, #Pueden perderse mensajes
        InitialDistinctValues 
    )
        
    def initializer(self):
        self.apply_restrictions()
        coordinator = self.network.nodes_sorted()[0]
        for node in self.network.nodes():
            node.memory["self"] = node
            node.memory["coordinator"] = coordinator
            node.memory["crash_chance"] = CrashBehavior.crash_chance(node)
            node.status = self.Status.PARTICIPANT
        coordinator.memory["oks"] = {}
        coordinator.memory["readys"] = {}
        coordinator.status = self.Status.COORDINATOR
        coordinator.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=coordinator))



    def crash(self, node : NodeAccess):
        self.logging.append(f"[Node {node.memory['unique_value']}] Crashed")
        if node.status != self.Status.COORDINATOR:
            self.transaction.crash()
        node.status = self.Status.CRASHED

    def send_to_participants(self, node : NodeAccess, header: str):
        for neighbor in node.neighbors():
            if CrashBehavior.determine_crash(node):
                self.crash(node)
                return
            self.logging.append(f"[Node {node.memory['unique_value']}] Sending '{header}' to [Node {neighbor}]")
            self.send(
                node, 
                data=None,
                destination=neighbor,
                header=header
            )

    @Status.COORDINATOR
    def spontaneously(self, node: NodeAccess, _: Message):
        self.logging.append(f"[Node {node.memory['unique_value']}] is the coordinator")
        m = Message(
            destination=node.memory["coordinator"],
            header=self.default_params["PrepareTimeout"]
            )
        node.memory["PrepareAlarm"] = self.set_alarm(node, self.waiting_time, m)
        self.send_to_participants(node, self.default_params["Prepare"])
        self.crash(node)
        # node.status = self.Status.POST_PREPARE_COORDINATOR



    @Status.POST_PREPARE_COORDINATOR
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["OK"]:
            node.memory["oks"][message.source] = True
            if len(node.memory["oks"]) == len(node.neighbors()):
                self.disable_alarm(node.memory["PrepareAlarm"])
                # Asumo que los nodos solo pueden fallar al principio. Una vez que confirman que pueden hacer la transaccion, no crashean
                # Entonces: no inicio ninguna alarma de nuevo espertando los readys
                self.send_to_participants(node, self.default_params["PreCommit"])
                node.status = self.Status.PRE_COMMIT_COORDINATOR
        else:
            error("POST_PREPARE_COORDINATOR::receiving", message)
        
    @Status.POST_PREPARE_COORDINATOR
    def alarm(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["PrepareTimeout"]:
            self.send_to_participants(node, self.default_params["Abort"])
            node.status = self.Status.SUCCESS
        else:
            error("POST_PREPARE_COORDINATOR::alarm", message)



    @Status.PRE_COMMIT_COORDINATOR
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["Ready"]:
            node.memory["readys"][message.source] = True
            if len(node.memory["readys"]) == len(node.neighbors()):
                self.send_to_participants(node, self.default_params["Commit"])
                node.status = self.Status.SUCCESS
        else:
            error("PRE_COMMIT_COORDINATOR::receiving", message)



    @Status.PARTICIPANT
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["Prepare"]:
            if CrashBehavior.determine_crash(node):
                self.crash(node)
                return
            m = Message(
                destination=node.memory["self"],
                header=self.default_params["PreCommitTimeout"]
                )
            node.memory["PreCommitAlarm"] = self.set_alarm(node, 2*self.waiting_time, m)
            self.send(
                node, 
                data=None,
                destination=node.memory["coordinator"],
                header=self.default_params["OK"]
            )
            node.status = self.Status.POST_PREPARE_PARTICIPANT
        else:
            error("PARTICIPANT::receiving", message)



    @Status.POST_PREPARE_PARTICIPANT
    def receiving(self, node: NodeAccess, message: Message):
        #Asumo que los participantes no fallan despues de confirmar que pueden hacer la transaccion
        self.disable_alarm(node.memory["PreCommitAlarm"])
        if message.header == self.default_params["PreCommit"]:
            m = Message(
                destination=node.memory["self"],
                header=self.default_params["DoCommitTimeout"]
                )
            node.memory["DoCommitAlarm"] = self.set_alarm(node, self.waiting_time, m)
            self.send(
                node, 
                data=None,
                destination=node.memory["coordinator"],
                header=self.default_params["Ready"]
            )
            node.status = self.Status.PRE_COMMIT_PARTICIPANT
        elif message.header == self.default_params["Abort"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Aborting transaction")
            self.transaction.abort()
            node.status = self.Status.SUCCESS
        else:
            error("POST_PREPARE_PARTICIPANT::receiving", message)

    @Status.POST_PREPARE_PARTICIPANT
    def alarm(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["PreCommitTimeout"]:
            # m = Message(
            #     destination=node.memory["self"],
            #     header=self.default_params["NeighborsRoundTimeout"]
            #     )
            # node.memory["NeighborsRoundAlarm"] = self.set_alarm(node, self.waiting_time, m)
            # self.send_to_participants(node, self.default_params["Request"])
            # node.status = self.Status.RECOVERING
            self.logging.append(f"[Node {node.memory['unique_value']}] Aborting transaction")
            self.transaction.abort()
            node.status = self.Status.SUCCESS
        else:
            error("POST_PREPARE_PARTICIPANT::alarm", message)



    @Status.PRE_COMMIT_PARTICIPANT
    def receiving(self, node: NodeAccess, message: Message):
        #Asumo que los participantes no fallan despues de confirmar que pueden hacer la transaccion
        self.disable_alarm(node.memory["DoCommitAlarm"])
        if message.header == self.default_params["Commit"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Committing transaction")
            self.transaction.commit()
            node.status = self.Status.SUCCESS
        elif message.header == self.default_params["Abort"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Aborting transaction")
            self.transaction.abort()
            node.status = self.Status.SUCCESS
        else:
            error("PRE_COMMIT_PARTICIPANT::receiving", message)


    @Status.PRE_COMMIT_PARTICIPANT
    def alarm(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["DoCommitTimeout"]:
            #Se que a los otros nodos tambien les debe haber llegado un precommit (asumiendo que no se pierden mensajes)
            self.logging.append(f"[Node {node.memory['unique_value']}] Committing transaction")
            self.transaction.commit()
            node.status = self.Status.SUCCESS
        else:
            error("PRE_COMMIT_PARTICIPANT::alarm", message)




    #TODO implement recovery
    @Status.RECOVERING
    def default(self, *args, **kwargs):    
        pass


    @Status.SUCCESS
    def default(self, *args, **kwargs):    
        pass

    @Status.CRASHED
    def default(self, *args, **kwargs):    
        pass


#### Pruebas

Forzar crash en momentos concretos:

In [ ]:
n = 5

sim.run(1)

for i in range(10):
    net_gen = NetworkGenerator(directed=False)
    net = net_gen.generate_complete_network(n)
    sim = Simulation(net, check_restrictions=True)
    sim.algorithms = ((ThreePhaseCommit, {}),)
    sim.reset()
    sim.run()
    alg = sim.algorithms[0]
    for log in alg.logging:
        logger.info(log)
    if alg.transaction.state == Transaction.TransactionState.INCONSISTENT:
        logger.info(f"Iteracion {i} perdió atomicidad")
    elif alg.transaction.state == Transaction.TransactionState.ONGOING:
        logger.info(f"Iteracion {i} terminó en deadlock")
    else:
        logger.info(f"Iteracion {i} terminó correctamente con estado {alg.transaction.state}")
    logger.info("\n")

### 3. Integración con escenarios bizantinos

- Modifique el protocolo para tolerar fallos bizantinos, incorporando firmas digitales y verificación de mensajes.

In [ ]:
#TODO


- Simule casos en los que un participante miente sobre su voto o altera mensajes.

In [ ]:
 #TODO